<center> 
    <h3><b> Adnan Rashid </b></h3>
    <nav>
    | <a href="00-Contents-Setup.ipynb"> Home Page </a> | 
    <a href="http://bit.ly/cka_notes_original"> Kubernetes Notes </a> |
    <a href="https://adnan.study"> Website </a> |
    <a href="https://www.instagram.com/adnans_techie_studies/"> Instagram </a> |
    <a href="https://www.linkedin.com/in/adnanrashid1/"> LinkedIn </a> |
    </nav>

</center>

<h1 style="background:black"><code style="background:black;color:white"><center>CI/CD with CodePipeline</center></code></h1>

<div class="alert alert-block alert-warning">
<font style="color:black">
    <center> Continous Integration (CI) and Continous Delivery (CD) are essential for companies.<br><br> Teams are more productive when they can make discrete changes frequently, release those changes programmatically and deliver updates without disruption.<br><br> In this module, we will build a CI/CD pipeline using AWS CodePipeline. The CI/CD pipeline will deploy a sample Kubernetes service, we will make a change to the GitHub repository and observe the automated delivery of this change to the cluster.  
</center>
</font>
</div>

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/32.jpg" width="50%" height="50%" border=5/></center>

## Pre-requisite

* Check your variables are loaded

In [ ]:
%store

* Load in variable store

In [ ]:
%store -r 

* Load variables into notebook

In [ ]:
%env AWS_PROFILE=$AWS_PROFILE
%env AWS_DEFAULT_REGION=$AWS_DEFAULT_REGION
%env ACCOUNT_ID=$ACCOUNT_ID

* Check identity 

In [ ]:
!aws sts get-caller-identity

<br>

---
---

<br><br>

<br>

## Create IAM Role

In an AWS CodePipeline, we are going to use AWS CodeBuild to deploy a sample Kubernetes service. 

This requires an AWS IAM role capable of interacting with the EKS cluster. 

In this step, we are going to create an IAM role and add an inline policy that we will use in the CodeBuild stage to interact with the EKS cluster via kubectl. 

* Create the role

In [ ]:
%%bash 
TRUST="{ \"Version\": \"2012-10-17\", \"Statement\": [ { \"Effect\": \"Allow\", \"Principal\": { \"AWS\": \"arn:aws:iam::${ACCOUNT_ID}:root\" }, \"Action\": \"sts:AssumeRole\" } ] }"
echo '{ "Version": "2012-10-17", "Statement": [ { "Effect": "Allow", "Action": "eks:Describe*", "Resource": "*" } ] }' > codepipeline/iam-role-policy

aws iam create-role --role-name EksWorkshopCodeBuildKubectlRole --assume-role-policy-document "$TRUST" --output text --query 'Role.Arn'

aws iam put-role-policy --role-name EksWorkshopCodeBuildKubectlRole --policy-name eks-describe --policy-document file://codepipeline/iam-role-policy

* Modify AWS-Auth ConfigMap 

Now that we have the IAM role created, we are going to add the role to the aws-auth ConfigMap for the EKS clsuter. 

Once the ConfigMap includes this new role, kubectl in the CodeBuild stage of the pipeline will be able to interact with the EKS cluster via the IAM role

In [ ]:
%%bash 

ROLE="    - rolearn: arn:aws:iam::$ACCOUNT_ID:role/EksWorkshopCodeBuildKubectlRole\n      username: build\n      groups:\n        - system:masters"

kubectl get -n kube-system configmap/aws-auth -o yaml | awk "/mapRoles: \|/{print;print \"$ROLE\";next}1" > codepipeline/aws-auth-patch.yml
kubectl patch configmap/aws-auth -n kube-system --patch "$(cat codepipeline/aws-auth-patch.yml)"

It is also possible to edit the aws-auth ConfigMap manually, you can run the following

<p style="background:black">
<code style="background:black;color:white">> kubectl edit -n kube-system configmap/aws-auth

We are going to fork the sample Kubernetes service so that we will be able to modify the repository and trigger builds. 

* Login to Github and fork the sample service to your own account 

## <br><br>Github Access Token

In order for CodePipeline to receive callbacks from GitHub, we need to generate a personal access token. 

Once created, an access token can be stored in a secure enclave and reused, so this step is only required during the first run or when you need to generate new keys. 

Open up the <a href="https://github.com/settings/tokens/new"> New personal access page </a> in GitHub and do following

* Enter a value for <b> Token Description </b> 
* Check the <b>repo</b> permission scope and scroll down and click the <b> Generate token </b> button 

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/github_token_name.png" width="70%" height="70%" border=5/>

* Copy the <b> personal access token </b> and save it in a secure plae for the next step

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/github_copy_access.png" width="70%" height="70%" border=5/>

## <br><br> CodePipeline Setup 

We are going to create the AWS CodePipeline using iAWS CloudFormation. 

CloudFormation is an Infrastructure as Code (IaC) tool which provides a common language for you to describe and provision all the infrastructure resources in your cloud enviornment. CloudFormation allows you to use a simple text file to model and provision, in an automated and secure manner, all the resources needed for your applications across all regions and accounts. 

Each EKS deployment/service should have its own CodePipeline and be located in an isoalted source repository. 

You can modify the CloudFormation template provided to meet your system requirements to easily onboard new services to your EKS cluster. For each new service the following steps can be repeated. 

* Deploy the following stacks by copying to browser

In [ ]:
https://console.aws.amazon.com/cloudformation/home?#/stacks/create/review?stackName=eksws-codepipeline&templateURL=https://s3.amazonaws.com/eksworkshop.com/templates/main/ci-cd-codepipeline.cfn.yml

* After the console is open, enter your Github username and personal access toekn (created earlier), check the acknowledge box and then click the 'Craete Stack' button located at the bottom of the page

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/cloudformation_stack.png" width="100%" height="100%" border=5/>

* Wait for the status to change from 'CREATE_IN_PROGRESS' to <b> CREATE_COMPLETE </b> before moving on to the next step. 

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/cloudformation_stack_creating.png" width="100%" height="100%" border=5/>

* Open <a href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines"> Codepipeline in the management console </a>. You will see a CodePipeline that starts with <b> eks-workshop-codepipeline </b>. 

Note: Ensure you are in the correct region where it was deployed

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/codepipeline_landing.png" width="100%" height="100%" border=5/>

* Once you are on the detail page for the specific CodePipeline, you can see the status along with the links to the change and build details. 

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/codepipeline_details.png" width="100%" height="100%" border=5/>

* To review that status of the deployment you can run the following

In [ ]:
!kubectl describe deployment hello-k8s

* For the status of the service, run following

In [ ]:
!kubectl describe service hello-k8s

* Once the service is built and delivered, we can run the following command to get the Elastic Load Balancer (ELB) endpoint and open it in a browser. 

If the message is not updated immediately, give Kubernetes some time to deploy the change 

In [ ]:
!kubectl get service hello-k8s -o wide

* Check the URL

In [ ]:
!curl a79bbc651d95e4c6e8fdb54b07aa77e5-995409802.eu-west-1.elb.amazonaws.com

## <br><br> Trigger New Release 

So far we have walked through setting up CI/CD for EKS using AWS CodePipeline and now we are going to make a change to the Github Repository so that we can see a new release built and delivered. 

* Clone the repository

Make a change to the <b> main.go </b> file and change the text where it says 'Hello World' 

* Before 

* After 

In [ ]:
func main() {
	http.HandleFunc("/", func(w http.ResponseWriter, r *http.Request) {

		f := fib()

		res := &response{Message: "New Message"}


* Commit and push the changes 

In [ ]:
git commit -am 'change the message' 
git push 

<br>
After you modify and commit the change in Github, in approximately 1 minute you will see a new build triggered in AWS 

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/codepipeline_building.png" width="100%" height="100%" border=5/>

* Lets check the website once the build is complete and you will notice at the top the message has changed

In [ ]:
!curl a79bbc651d95e4c6e8fdb54b07aa77e5-995409802.eu-west-1.elb.amazonaws.com

## <br><br> Clean up 

* Delete the Kubernetes deployment and service 

In [ ]:
!kubectl delete deployments hello-k8s
!kubectl delete services hello-k8s

* Delete the Cloudformation stack created in the console 

In [ ]:
!aws cloudformation delete-stack --stack-name eksws-codepipeline

* Delete the ECR Repository find your repo first

In [ ]:
!aws ecr describe-repositories --output "text"

In [ ]:
!aws ecr delete-repository --repository-name ecrdockerrepository-27eh2ehcjlka --force

* Delete S3 bucket 

In [ ]:
!aws s3 delete-bucket --bucket-name eksws-codepipeline*

In [ ]:
!aws s3 ls | sort

* Remove contents from bucket 

In [ ]:
!aws s3 rm s3://eksws-codepipeline-codepipelineartifactbucket-1n3alcqr9so8k --recursive 

In [ ]:
!aws s3api delete-bucket --bucket eksws-codepipeline-codepipelineartifactbucket-1n3alcqr9so8k